In [1]:
import oemof.solph as solph

# Create an energy system model
model = solph.Model()

# Define the time horizon for the optimization (e.g., 24 hours)
hours = range(24)

# Create the electricity bus (grid)
electricity_bus = solph.Bus(label='electricity_bus')
model.add(electricity_bus)

# Create the PV converter
pv_converter = solph.Source(
    label='pv_converter',
    outputs={electricity_bus: solph.Flow(nominal_value=10e3, variable_costs=0)}
)
model.add(pv_converter)

# Create the wind energy converter
wind_converter = solph.Source(
    label='wind_converter',
    outputs={electricity_bus: solph.Flow(nominal_value=20e3, variable_costs=0)}
)
model.add(wind_converter)

# Create the storage system
storage = solph.components.GenericStorage(
    label='storage',
    inputs={
        electricity_bus: solph.Flow(nominal_value=10e3, variable_costs=0)
    },
    outputs={
        electricity_bus: solph.Flow(nominal_value=10e3, variable_costs=0)
    },
    nominal_storage_capacity=20e3,
    inflow_conversion_factor=0.95,
    outflow_conversion_factor=0.95,
    loss_rate=0.01,
    initial_storage_level=None  # Can be set to a specific value if desired
)
model.add(storage)

# Create the electric load
electric_load = solph.Sink(
    label='electric_load',
    inputs={electricity_bus: solph.Flow()}
)
model.add(electric_load)

# Define the objective function
model.objective = solph.Minimize(solph.LinearExpression(
    sum(flow.variable_costs.sum() for flow in model.flows)
))

# Optimize the model
solver = solph.Solver()
results = solver.solve(model)

# Access the optimized results
energies = results[model.groups[0]]

# Print the optimized energy flows
for hour in hours:
    print(f"Hour {hour}:")
    for flow in energies:
        print(f"{flow}: {energies[flow][hour]}")


ModuleNotFoundError: No module named 'oemof'